<a href="https://colab.research.google.com/github/Aalezz/ine-Tuning-CodeT5-for-Python-Bug-Fixing-with-LoRA-and-Prompt-Tuning/blob/main/CodeT5_BugFixing_100_Samples_LoRA_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Modified for 100-Sample LoRA Fine-Tuning

This notebook has been automatically adapted to fine-tune CodeT5 using LoRA on the `curated_pytracebugs_subset_100.jsonl` (100 samples).

**Key Changes:**
- **Dataset:** Configured to use `curated_pytracebugs_subset_100.jsonl`. You will be prompted to upload this file.
- **Output Directories:**
    - Main output: `/content/codet5_lora_bugfix_100_samples`
    - LoRA Adapters: `/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample`
- **Training Parameters (Optimized for 100 samples):**
    - `per_device_train_batch_size`: 4
    - `gradient_accumulation_steps`: 1
    - `num_train_epochs`: 5
    - `logging_steps`: 5
    - `save_steps`: 10
    - `learning_rate`: 5e-5
- **Prompt Template:** Set to `"Fix the following Python code: {buggy_code}"`
- **Max Token Lengths:** `MAX_TOKEN_LENGTH` set to 256, `GENERATION_MAX_LENGTH` to 150.
- Data augmentation sections (if any) from the original notebook are not used as we are using a pre-defined curated dataset.

**Please ensure `curated_pytracebugs_subset_100.jsonl` is uploaded when prompted.**


# CodeT5 Fine-Tuning for Python Bug Fixing with LoRA

This notebook demonstrates how to fine-tune the CodeT5 model for automated Python bug fixing using Prompt Tuning, Data Augmentation (with a synthetic dataset), and LoRA (Low-Rank Adaptation) with the PEFT library. The model is trained and evaluated on buggy/fixed Python code pairs, using BLEU score and exact match accuracy as metrics.

## 1. Setup Environment

First, we need to install the necessary Python libraries.

In [1]:
!pip install -q torch transformers==4.41.2 peft==0.10.0 accelerate bitsandbytes evaluate nltk sacrebleu datasets sentencepiece

## 2. Import Libraries

In [2]:
import json
import os
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import torch
import evaluate
from google.colab import files # For file uploads
import shutil # For unzipping

## 3. Configuration and Device Setup

In [3]:
# Ensure GPU is available if possible, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configuration variables
MODEL_NAME = "Salesforce/codet5-base"
# SYNTHETIC_DATASET_FILENAME = "synthetic_buggy_fixed_pairs.json" # Commented out by script
SYNTHETIC_DATASET_FILENAME = "curated_pytracebugs_subset_100.jsonl" # Updated for 100 samples
# LORA_ADAPTERS_ZIP_FILENAME = "codet5_lora_adapters.zip" # Optional, if you want to load pre-trained adapters # Commented out by script
LORA_ADAPTERS_ZIP_FILENAME = "codet5_lora_adapters_100_sample.zip" # Optional for 100 samples
OUTPUT_DIR = "/content/codet5_lora_bugfix_100_samples" # Updated for 100 samples
LORA_OUTPUT_DIR = "/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample" # Updated for 100 samples
MAX_TOKEN_LENGTH = 256
GENERATION_MAX_LENGTH = 150
PROMPT_TEMPLATE = "Fix the following Python code: {buggy_code}"


Using device: cpu


## 4. Data Preparation

You need to upload the `synthetic_buggy_fixed_pairs.json` file. This file contains the small dataset of buggy and fixed Python code snippets.

In [4]:
DATASET_FILENAME_EXPECTED = "curated_pytracebugs_subset_100.jsonl" # Expected name for the 100-sample dataset
print(f"Please upload the '{DATASET_FILENAME_EXPECTED}' file.")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Please upload the dataset to proceed.")
    DATASET_PATH = None
else:
    uploaded_filename = next(iter(uploaded))
    DATASET_PATH = f"/content/{uploaded_filename}" # This is the crucial variable for the rest of the notebook
    print(f"File '{uploaded_filename}' uploaded successfully and will be used from: {DATASET_PATH}")
    if uploaded_filename != DATASET_FILENAME_EXPECTED:
        print(f"Warning: Uploaded file name '{uploaded_filename}' differs from expected '{DATASET_FILENAME_EXPECTED}'. Using the uploaded file name.")

    # Update SYNTHETIC_DATASET_FILENAME as well, in case other parts of the notebook use it, though DATASET_PATH is primary.
    # This assumes SYNTHETIC_DATASET_FILENAME is defined and used as a global or accessible variable.
    # For safety, ensure it's explicitly set if needed, or refactor notebook to only use DATASET_PATH.
    # For this script, we'll assume DATASET_PATH is the main one used after this cell.


Please upload the 'curated_pytracebugs_subset_100.jsonl' file.


Saving curated_pytracebugs_subset_100.jsonl to curated_pytracebugs_subset_100 (1).jsonl
File 'curated_pytracebugs_subset_100 (1).jsonl' uploaded successfully and will be used from: /content/curated_pytracebugs_subset_100 (1).jsonl


### 4.1. (Optional) Upload Pre-trained LoRA Adapters

If you have already trained LoRA adapters (e.g., the `codet5_lora_adapters.zip` file I provided) and want to skip training and go directly to evaluation, upload the zip file here.

In [5]:
UPLOAD_PRETRAINED_ADAPTERS = False  # Set to True if you want to upload and use pre-trained adapters

if UPLOAD_PRETRAINED_ADAPTERS:
    print(f"Please upload the '{LORA_ADAPTERS_ZIP_FILENAME}' file.")
    uploaded_adapters = files.upload()
    if LORA_ADAPTERS_ZIP_FILENAME in uploaded_adapters:
        print(f"'{LORA_ADAPTERS_ZIP_FILENAME}' uploaded successfully!")
        # Unzip to the expected LORA_OUTPUT_DIR
        os.makedirs(LORA_OUTPUT_DIR, exist_ok=True)
        shutil.unpack_archive(LORA_ADAPTERS_ZIP_FILENAME, LORA_OUTPUT_DIR, 'zip')
        print(f"Adapters unzipped to {LORA_OUTPUT_DIR}")
    else:
        print(f"Error: '{LORA_ADAPTERS_ZIP_FILENAME}' not found. Proceeding without pre-trained adapters (will train from scratch if training is run).")
else:
    print("Skipping upload of pre-trained adapters. Will train from scratch if training is run.")


Skipping upload of pre-trained adapters. Will train from scratch if training is run.


## 5. Training Script Components

These are the components from `train_codet5_lora.py` adapted for Colab.

In [6]:
class BugFixDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data_path, prompt_template, max_length=MAX_TOKEN_LENGTH):
        self.tokenizer = tokenizer
        self.prompt_template = prompt_template
        self.max_length = max_length

        if data_path is None or not os.path.exists(data_path):
            raise FileNotFoundError(f"Dataset file not found at {data_path}. Please upload it first.")
               # ... (other parts of __init__)
        self.data = []
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip(): # Ensure line is not empty
                    try:
                        self.data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON from line: {line.strip()}")
                        print(f"Error: {e}")
                        # Optionally, you might want to skip corrupted lines or raise an error
                        # For now, this will print an error and potentially lead to an empty self.data or partial data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        buggy_code = item["buggy_code"]
        fixed_code = item["fixed_code"]

        input_text = self.prompt_template.format(buggy_code=buggy_code)

        model_inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                fixed_code,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            ).input_ids

        labels[labels == self.tokenizer.pad_token_id] = -100

        model_inputs["labels"] = labels.squeeze()
        model_inputs["input_ids"] = model_inputs["input_ids"].squeeze()
        if "attention_mask" in model_inputs:
             model_inputs["attention_mask"] = model_inputs["attention_mask"].squeeze()

        return model_inputs

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

### 5.1. Training Function

In [7]:
def train_model():
    if DATASET_PATH is None:
        print("Cannot train: Dataset path is not set. Please upload the dataset first.")
        return

    print(f"Starting training with dataset: {DATASET_PATH}")
    tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
    base_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q", "v"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM
    )
    peft_model = get_peft_model(base_model, lora_config)
    print("PEFT Model with LoRA for Training:")
    print_trainable_parameters(peft_model)
    peft_model.print_trainable_parameters()

    train_dataset = BugFixDataset(tokenizer, DATASET_PATH, PROMPT_TEMPLATE)
    eval_dataset = BugFixDataset(tokenizer, DATASET_PATH, PROMPT_TEMPLATE)  # Using same for eval

    training_args = Seq2SeqTrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=4,  # Modified by script
        gradient_accumulation_steps=1,  # Modified by script
        per_device_eval_batch_size=1,
        num_train_epochs=5,  # Modified by script
        logging_steps=5,  # Modified by script
        save_steps=10,  # Modified by script
        eval_strategy="epoch",
        learning_rate=5e-5,  # Modified by script
        weight_decay=0.01,  # Modified by script
        warmup_steps=0,  # Modified by script
        lr_scheduler_type="linear",  # Modified by script
        report_to="tensorboard",  # Modified by script
        save_total_limit=1,  # Modified by script
        fp16=torch.cuda.is_available(),  # Modified by script
        remove_unused_columns=False,  # Modified by script
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model)

    trainer = Seq2SeqTrainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\n--- Starting Training ---")
    trainer.train()
    print("--- Training Complete ---")

    os.makedirs(LORA_OUTPUT_DIR, exist_ok=True)
    print(f"Saving LoRA adapters to {LORA_OUTPUT_DIR}")
    peft_model.save_pretrained("/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample")  # Path updated by script
    tokenizer.save_pretrained("/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample")  # Path updated by script
    print("LoRA adapters and tokenizer saved.")


## 6. Evaluation Script Components

These are the components from `evaluate_models.py` adapted for Colab.

In [8]:
def generate_predictions(model, tokenizer, dataset_items, prompt_template, max_length=MAX_TOKEN_LENGTH, generation_max_length=GENERATION_MAX_LENGTH):
    predictions = []
    references = []
    for item in dataset_items:
        buggy_code = item["buggy_code"]
        fixed_code = item["fixed_code"]

        input_text = prompt_template.format(buggy_code=buggy_code)
        input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=max_length).input_ids.to(device)

        try:
            generation_config = GenerationConfig(
                max_length=generation_max_length,
                num_beams=5,
                early_stopping=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                decoder_start_token_id=model.config.decoder_start_token_id if hasattr(model.config, "decoder_start_token_id") else tokenizer.bos_token_id
            )
            generated_ids = model.generate(input_ids=input_ids, generation_config=generation_config)
            generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        except Exception as e:
            print(f"Error during generation: {e}")
            generated_code = ""

        predictions.append(generated_code)
        references.append(fixed_code)
    return predictions, references

def calculate_metrics(predictions, references):
    exact_match_metric = evaluate.load("exact_match")
    sacrebleu_metric = evaluate.load("sacrebleu")

    references_for_sacrebleu = [[ref] for ref in references]

    em_results = exact_match_metric.compute(predictions=predictions, references=references)
    sacrebleu_results = sacrebleu_metric.compute(predictions=predictions, references=references_for_sacrebleu)

    return {
        "exact_match": em_results["exact_match"],
        "sacrebleu": sacrebleu_results["score"]
    }

### 6.1. Evaluation Function

In [9]:
def evaluate_models():
    if DATASET_PATH is None:
        print("Cannot evaluate: Dataset path is not set. Please upload the dataset first.")
        return

    # Load JSONL data line-by-line
    evaluation_data_list = []
    with open(DATASET_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    evaluation_data_list.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from line during evaluation: {line.strip()}")
                    print(f"Error: {e}")

    tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

    # 1. Evaluate Base Model
    print("--- Evaluating Base Model --- ")
    base_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)

    base_predictions, base_references = generate_predictions(
        base_model, tokenizer, evaluation_data_list, PROMPT_TEMPLATE
    )
    base_metrics = calculate_metrics(base_predictions, base_references)
    print(f"Base Model Metrics: {base_metrics}")
    for i in range(len(evaluation_data_list)):
        print(f"  Example {i+1}:")
        buggy_code_to_print = evaluation_data_list[i]["buggy_code"].replace('\n', '\\n')
        print(f"    Buggy: {buggy_code_to_print}")
        fixed_code_to_print = base_references[i].replace('\n', '\\n')
        print(f"    Fixed (Reference): {fixed_code_to_print}")
        base_prediction_to_print = base_predictions[i].replace('\n', '\\n')
        print(f"    Base Model Prediction: {base_prediction_to_print}")

    del base_model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 2. Evaluate Fine-tuned Model (with LoRA)
    print("\n--- Evaluating Fine-tuned LoRA Model --- ")
    if not os.path.exists("/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample") or not os.listdir("/content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample"):
        print(f"LoRA adapters not found at {LORA_OUTPUT_DIR}. Please train the model first or upload pre-trained adapters.")
        return

    peft_config = PeftConfig.from_pretrained(LORA_OUTPUT_DIR)
    tuned_base_model = T5ForConditionalGeneration.from_pretrained(peft_config.base_model_name_or_path).to(device)
    tuned_model = PeftModel.from_pretrained(tuned_base_model, LORA_OUTPUT_DIR).to(device)
    tuned_model.eval()

    tuned_predictions, tuned_references = generate_predictions(
        tuned_model, tokenizer, evaluation_data_list, PROMPT_TEMPLATE
    )
    tuned_metrics = calculate_metrics(tuned_predictions, tuned_references)
    print(f"Fine-tuned LoRA Model Metrics: {tuned_metrics}")
    for i in range(len(evaluation_data_list)):
        print(f"  Example {i+1}:")
        buggy_code_to_print = evaluation_data_list[i]["buggy_code"].replace('\n', '\\n')
        fixed_code_to_print = tuned_references[i].replace('\n', '\\n')
        tuned_prediction_to_print = tuned_predictions[i].replace('\n', '\\n')
        print(f"    Buggy: {buggy_code_to_print}")
        print(f"    Fixed (Reference): {fixed_code_to_print}")
        print(f"    Tuned Model Prediction: {tuned_prediction_to_print}")

    print("\n--- Evaluation Summary ---")
    print(f"Base Model ({MODEL_NAME}): Exact Match: {base_metrics['exact_match']:.4f}, SacreBLEU: {base_metrics['sacrebleu']:.2f}")
    print(f"Fine-tuned CodeT5 + LoRA: Exact Match: {tuned_metrics['exact_match']:.4f}, SacreBLEU: {tuned_metrics['sacrebleu']:.2f}")


## 7. Run Workflow

Choose whether to train the model or just run evaluation (if you uploaded pre-trained adapters).

In [10]:
# Set to True to run training. If False, it will skip to evaluation (assuming adapters are present).
RUN_TRAINING = True

if RUN_TRAINING:
    print("Proceeding with training...")
    train_model()
    print("Training finished. Now proceeding to evaluation.")
    evaluate_models()
else:
    print("Skipping training. Proceeding directly to evaluation.")
    if os.path.exists(LORA_OUTPUT_DIR) and os.listdir(LORA_OUTPUT_DIR):
        evaluate_models()
    else:
        print(f"Cannot evaluate: LoRA adapters not found at {LORA_OUTPUT_DIR} and training was skipped. Please upload adapters or set RUN_TRAINING to True.")


Proceeding with training...
Starting training with dataset: /content/curated_pytracebugs_subset_100 (1).jsonl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


PEFT Model with LoRA for Training:
trainable params: 884736 || all params: 223766784 || trainable%: 0.40
trainable params: 884,736 || all params: 223,766,784 || trainable%: 0.3953830788397978

--- Starting Training ---


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,5.202700,4.906707
2,4.943700,4.103770
3,4.350800,3.538397
4,4.058200,3.218665
5,3.574800,3.115981


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-pac

--- Training Complete ---
Saving LoRA adapters to /content/codet5_lora_bugfix_100_samples/lora_adapters_100_sample


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LoRA adapters and tokenizer saved.
Training finished. Now proceeding to evaluation.
--- Evaluating Base Model --- 


Base Model Metrics: {'exact_match': np.float64(0.0), 'sacrebleu': 1.28290137611128}
  Example 1:
    Buggy:     def _add_datetimelike_methods(cls):\n        """\n        add in the datetimelike methods (as we may have to override the\n        superclass)\n        """\n\n        def __add__(self, other):\n            from pandas.core.index import Index\n            from pandas.core.indexes.timedeltas import TimedeltaIndex\n            from pandas.tseries.offsets import DateOffset\n            if isinstance(other, TimedeltaIndex):\n                return self._add_delta(other)\n            elif isinstance(self, TimedeltaIndex) and isinstance(other, Index):\n                if hasattr(other, '_add_delta'):\n                    return other._add_delta(self)\n                raise TypeError("cannot add TimedeltaIndex and {typ}"\n                                .format(typ=type(other)))\n            elif isinstance(other, (DateOffset, timedelta, np.timedelta64)):\n                return self

## 8. (Optional) Download Trained Adapters

If you ran the training, you can download the LoRA adapters for later use.

In [ ]:
DOWNLOAD_ADAPTERS = False # Set to True if you want to download the adapters after training

if RUN_TRAINING and DOWNLOAD_ADAPTERS:
    if os.path.exists(LORA_OUTPUT_DIR):
        print(f"Zipping adapters from {LORA_OUTPUT_DIR} for download...")
        shutil.make_archive(f"/content/{LORA_ADAPTERS_ZIP_FILENAME}", 'zip', LORA_OUTPUT_DIR)
        files.download(f"/content/{LORA_ADAPTERS_ZIP_FILENAME}")
        print(f"'{LORA_ADAPTERS_ZIP_FILENAME}' ready for download.")
    else:
        print(f"Adapters directory {LORA_OUTPUT_DIR} not found. Cannot download.")
elif DOWNLOAD_ADAPTERS:
    print("Skipped downloading adapters because training was not run or DOWNLOAD_ADAPTERS is False.")